In [1]:
import yaml
import os
import pickle
import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm
from glob import glob

In [2]:
from ultralytics import YOLO

In [3]:
CONFIG_PATH = r'../config/config_sumo3.yaml'
DEVICE = 0
CAM = '02'
HALF = 0 if DEVICE == 0 or DEVICE == 2 else 1

In [4]:
with open(CONFIG_PATH, "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [5]:
camera_cfg = config[f'camera-{CAM}']
camera_cfg

{'annotations': '/data/public/svo0175/SmartHarbourDL/annotations/project_camera_02_09_to_11_06_23-2023_07_17_13_14_54-cvat for video 1.1.xml',
 'data-folder': '/data/public/svo0175/SmartHarbour/Camera_02'}

In [6]:
files = glob(f"{camera_cfg['data-folder']}/*.mkv")

In [7]:
for f in files:
    bname = os.path.basename(f)
    bname_no_ext = bname[:-4]

In [8]:
video_file = r'cfg_raw_cam_02_fhd_h265_20230609T090003.mkv'
video_file_name = video_file
video_file = os.path.join(camera_cfg['data-folder'], video_file)
video_file

'/data/public/svo0175/SmartHarbour/Camera_02/cfg_raw_cam_02_fhd_h265_20230609T090003.mkv'

In [9]:
model = YOLO("../saved_models/yolov8x.pt")  # load a pretrained model (recommended for training)

In [10]:
results = model(video_file, stream=True, device=3, imgsz=1920, save_txt=True, save_conf=True, save=True, classes=[8])  # predict on an image

In [11]:
res = []
res_raw = []
for result in results:
    curr = result.boxes.data.cpu().numpy()

    res.append(curr)
    res_raw.append(result.boxes)


video 1/1 (1/14978) /data/public/svo0175/SmartHarbour/Camera_02/cfg_raw_cam_02_fhd_h265_20230609T090003.mkv: 1088x1920 (no detections), 73.0ms
video 1/1 (2/14978) /data/public/svo0175/SmartHarbour/Camera_02/cfg_raw_cam_02_fhd_h265_20230609T090003.mkv: 1088x1920 (no detections), 70.9ms
video 1/1 (3/14978) /data/public/svo0175/SmartHarbour/Camera_02/cfg_raw_cam_02_fhd_h265_20230609T090003.mkv: 1088x1920 (no detections), 70.4ms
video 1/1 (4/14978) /data/public/svo0175/SmartHarbour/Camera_02/cfg_raw_cam_02_fhd_h265_20230609T090003.mkv: 1088x1920 (no detections), 70.2ms
video 1/1 (5/14978) /data/public/svo0175/SmartHarbour/Camera_02/cfg_raw_cam_02_fhd_h265_20230609T090003.mkv: 1088x1920 (no detections), 70.0ms
video 1/1 (6/14978) /data/public/svo0175/SmartHarbour/Camera_02/cfg_raw_cam_02_fhd_h265_20230609T090003.mkv: 1088x1920 (no detections), 70.1ms
video 1/1 (7/14978) /data/public/svo0175/SmartHarbour/Camera_02/cfg_raw_cam_02_fhd_h265_20230609T090003.mkv: 1088x1920 (no detections), 70.2m

In [12]:
result.boxes

WARNING ⚠️ 'Boxes.boxes' is deprecated. Use 'Boxes.data' instead.


ultralytics.engine.results.Boxes object with attributes:

boxes: tensor([], device='cuda:0', size=(0, 6))
cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))

In [13]:
result.boxes.data.cpu().numpy()

array([], shape=(0, 6), dtype=float32)

In [ ]:
with open(f'./output/{video_file_name}.pickle', 'wb') as handle:
    pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)